In [4]:
from abc import ABC, abstractmethod
from enum import Enum

# Define the QueryType enum
class QueryType(Enum):
    TECHNICAL = 1
    BILLING = 2
    GENERAL = 3

# Handler Interface
class SupportHandler(ABC):
    def __init__(self):
        self.next_handler = None

    def set_next_handler(self, next_handler):
        self.next_handler = next_handler

    @abstractmethod
    def handle_request(self, query_type, message):
        pass

# Concrete Handlers
class TechnicalSupportHandler(SupportHandler):
    def handle_request(self, query_type, message):
        if query_type == QueryType.TECHNICAL:
            print("Technical Support: Handling query - " + message)
        elif self.next_handler is not None:
            self.next_handler.handle_request(query_type, message)

class BillingSupportHandler(SupportHandler):
    def handle_request(self, query_type, message):
        if query_type == QueryType.BILLING:
            print("Billing Support: Handling query - " + message)
        elif self.next_handler is not None:
            self.next_handler.handle_request(query_type, message)

class GeneralSupportHandler(SupportHandler):
    def handle_request(self, query_type, message):
        if query_type == QueryType.GENERAL:
            print("General Support: Handling query - " + message)
        elif self.next_handler is not None:
            self.next_handler.handle_request(query_type, message)

# Client
class SupportChain:
    @staticmethod
    def get_support_chain():
        technical_support = TechnicalSupportHandler()
        billing_support = BillingSupportHandler()
        general_support = GeneralSupportHandler()

        technical_support.set_next_handler(billing_support)
        billing_support.set_next_handler(general_support)

        return technical_support


In [5]:
# Usage
if __name__ == "__main__":
    support_chain = SupportChain.get_support_chain()
    support_chain.handle_request(QueryType.TECHNICAL, "I can't connect to the internet.")
    support_chain.handle_request(QueryType.BILLING, "I have a question about my invoice.")
    support_chain.handle_request(QueryType.GENERAL, "Thank you for your service.")


Technical Support: Handling query - I can't connect to the internet.
Billing Support: Handling query - I have a question about my invoice.
General Support: Handling query - Thank you for your service.


In [6]:
from abc import ABC, abstractmethod

# Define the Email class
class Email:
    def __init__(self, sender, recipient, subject, body):
        self.sender = sender
        self.recipient = recipient
        self.subject = subject
        self.body = body
        self.is_spam = False
        self.has_virus = False
        self.is_moderated = True  # Assume it's moderated unless flagged

# Handler Interface
class EmailHandler(ABC):
    def __init__(self):
        self.next_handler = None

    def set_next_handler(self, handler):
        self.next_handler = handler

    @abstractmethod
    def handle(self, email: Email):
        pass

# Concrete Handlers
class SpamFilterHandler(EmailHandler):
    def handle(self, email: Email):
        spam_keywords = ['win money', 'free prize', 'click here']
        if any(keyword in email.subject.lower() or keyword in email.body.lower() for keyword in spam_keywords):
            email.is_spam = True
            print(f"SpamFilterHandler: Email marked as spam from {email.sender}")
            # Optionally, stop processing further handlers
        else:
            print("SpamFilterHandler: Email passed spam filter")
            if self.next_handler:
                self.next_handler.handle(email)

class VirusScanHandler(EmailHandler):
    def handle(self, email: Email):
        # Simulate virus scanning
        virus_signatures = ['malware', 'virus', 'trojan']
        if any(signature in email.body.lower() for signature in virus_signatures):
            email.has_virus = True
            print(f"VirusScanHandler: Virus detected in email from {email.sender}")
            # Optionally, stop processing further handlers
        else:
            print("VirusScanHandler: No virus detected")
            if self.next_handler:
                self.next_handler.handle(email)

class ContentModerationHandler(EmailHandler):
    def handle(self, email: Email):
        # Simulate content moderation
        inappropriate_content = ['hate speech', 'illegal activities', 'confidential']
        if any(content in email.body.lower() for content in inappropriate_content):
            email.is_moderated = False
            print(f"ContentModerationHandler: Email content flagged from {email.sender}")
            # Optionally, stop processing further handlers
        else:
            print("ContentModerationHandler: Email content is appropriate")
            if self.next_handler:
                self.next_handler.handle(email)

class DeliveryHandler(EmailHandler):
    def handle(self, email: Email):
        if not email.is_spam and not email.has_virus and email.is_moderated:
            print(f"DeliveryHandler: Email delivered to {email.recipient}")
        else:
            print("DeliveryHandler: Email not delivered due to previous flags")

# Client
class EmailProcessor:
    @staticmethod
    def get_email_processing_chain() -> EmailHandler:
        spam_filter = SpamFilterHandler()
        virus_scan = VirusScanHandler()
        content_moderation = ContentModerationHandler()
        delivery = DeliveryHandler()

        # Setting up the chain
        spam_filter.set_next_handler(virus_scan)
        virus_scan.set_next_handler(content_moderation)
        content_moderation.set_next_handler(delivery)

        return spam_filter


In [ ]:
# Usage
if __name__ == "__main__":
    # Create the processing chain
    processing_chain = EmailProcessor.get_email_processing_chain()

    # Example Emails
    emails = [
        Email(
            sender="spammer@example.com",
            recipient="user@example.com",
            subject="Win money now!",
            body="Click here to claim your free prize."
        ),
        Email(
            sender="friend@example.com",
            recipient="user@example.com",
            subject="Meeting Schedule",
            body="Let's meet tomorrow to discuss the project."
        ),
        Email(
            sender="malware@example.com",
            recipient="user@example.com",
            subject="Important Update",
            body="Please download the attached file to update your system. It contains malware."
        ),
        Email(
            sender="manager@example.com",
            recipient="user@example.com",
            subject="Confidential Report",
            body="Please find the confidential report attached."
        ),
    ]

    # Process each email
    for email in emails:
        print("\nProcessing new email:")
        processing_chain.handle(email)



Processing new email:
SpamFilterHandler: Email marked as spam from spammer@example.com

Processing new email:
SpamFilterHandler: Email passed spam filter
VirusScanHandler: No virus detected
ContentModerationHandler: Email content is appropriate
DeliveryHandler: Email delivered to user@example.com

Processing new email:
SpamFilterHandler: Email passed spam filter
VirusScanHandler: Virus detected in email from malware@example.com

Processing new email:
SpamFilterHandler: Email passed spam filter
VirusScanHandler: No virus detected
ContentModerationHandler: Email content flagged from manager@example.com


: 